In [1]:
import glob
import numpy as np
import cv2
# for chessboard
# root_origin = '/Users/scenery/Code/SomeGit/Stereo/Problems/stereo/left/'
root_origin = '/Users/scenery/Downloads/left/'
root_corner = '/Users/scenery/Downloads/corner/'
# root_corner = '/Users/scenery/Code/SomeGit/Stereo/Problems/stereo/corner/'

In [2]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

# images = sorted(glob.glob(root_origin + '*.jpg'))
images = sorted(glob.glob(root_origin + '*.jpeg'))

for i, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,6), corners2,ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)
        cv2.imwrite(root_corner + 'image%d.jpg' % i, img)
    
    if ret == False:
        print('%d image is false' % i)

0 image is false


In [31]:
#calibrateCamera(objectPoints, imagePoints, imageSize, cameraMatrix, distCoeffs[, rvecs[, tvecs[, flags[, criteria]]]]) → retval, cameraMatrix, distCoeffs, rvecs, tvecs
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, imageSize = gray.shape[::-1],None,None)
#wonder the meaning of retval and some input parameters

In [32]:
ret

0.15536906910078763

In [33]:
mtx

array([[534.07088367,   0.        , 341.53407538],
       [  0.        , 534.11914599, 232.94565269],
       [  0.        ,   0.        ,   1.        ]])

In [34]:
dist

array([[-2.92971637e-01,  1.07706963e-01,  1.31038377e-03,
        -3.11018871e-05,  4.34798099e-02]])

In [35]:
rvecs

[array([[-0.43239599],
        [ 0.25603401],
        [-3.08832021]]), array([[ 0.41531697],
        [ 0.65664497],
        [-1.3373494 ]]), array([[-0.2645143 ],
        [-0.39360849],
        [-2.74787379]]), array([[-0.37843358],
        [-0.18064237],
        [-3.11615996]]), array([[-0.45993978],
        [-0.3142018 ],
        [-1.76122223]]), array([[-0.29979221],
        [ 0.39216377],
        [-1.4348239 ]]), array([[-0.32034625],
        [ 0.1597993 ],
        [-1.24149595]]), array([[-0.45883216],
        [-0.08848877],
        [-1.33510786]]), array([[-0.35367631],
        [-0.24363035],
        [-1.56874295]]), array([[ 0.30697385],
        [ 0.5038552 ],
        [-1.82824733]]), array([[-0.17288944],
        [-0.46764681],
        [ 1.34745198]])]

In [36]:
tvecs

[array([[ 3.79739147],
        [ 0.89895017],
        [14.8593055 ]]), array([[-2.12419184],
        [ 1.84550818],
        [12.81229188]]), array([[3.03685642],
        [2.74656587],
        [9.90005138]]), array([[ 2.82321756],
        [ 2.22374307],
        [10.95762951]]), array([[-1.13991522],
        [ 2.67424464],
        [ 9.56289277]]), array([[ 1.60364764],
        [ 3.81296375],
        [16.12054474]]), array([[-5.94286694],
        [ 2.39629426],
        [16.77859026]]), array([[-3.27916485],
        [ 2.15811689],
        [11.72833343]]), array([[-2.89985494],
        [ 2.62066993],
        [10.57699258]]), array([[-1.22339632],
        [ 3.52139948],
        [15.6731179 ]]), array([[ 1.81888151],
        [-4.2642919 ],
        [12.45728517]])]

In [47]:
img = cv2.imread(root_origin + 'left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

In [44]:
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult1.png',dst)

True

In [46]:
# test of non-roi
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
cv2.imwrite('calibresult2.png',dst)

True

In [48]:
# test of not new camera matrix
dst = cv2.undistort(img, mtx, dist, None, None)
cv2.imwrite('calibresult3.png',dst)

True